In [ ]:
#!pip install learn2learn
#!pip install torchmeta
import torch
import utils
import model as MODEL
import train as TRAIN
import matplotlib.pyplot as plt
import epoch
from google.colab import drive
import epoch
import random
drive.mount('/content/drive')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if __name__ == '__main__':
    Quincy = False ## set true to run Quincy's model
    Alex = False
    if Quincy ==True: ## param search in this file doe snot support this 
        Params = {'datasize':96,'nways':32,'kshots':1,'in_channels':3,
                'num_classes':45,'hidden_size':2, "innerStep":.0006,
                'MetaLR':.0006,"number_of_tasks":1, "Order":True,"outerVSinner": 1,
                "epoch":50,'aug':True, 'trainvalSplit':.2,'traintestSplit': .8, 'Quincy':True, 'Alex':False}
    elif Alex == True: ### only run the CNN with augs
        Params = {'datasize':96,'nways':32,'kshots':1,'in_channels':3,
                'num_classes':45,'hidden_size':2, "innerStep":0.0001, ## <- Lr right there
                'MetaLR':.0006,"number_of_tasks":1, "Order":True,"outerVSinner": 1,
                "epoch":50,'aug':True, 'trainvalSplit':.2,'traintestSplit': .8, 'Quincy':False, 'Alex':True}
    else:
        Params = {'datasize': 96, 'nways': 5, 'kshots': 1, 'in_channels': 3, 'num_classes': 45,
                  'hidden_size': 2, 'innerStep': 0.001, 'MetaLR': 0.0005, 'number_of_tasks': 2,
                  'Order': True, 'outerVSinner': 1, 'epoch': 50, 'aug': True, 'Img_size': 96, 
                  'bs': 32, 'c_dim': 45, 'trainsz': 4725, 'trainvalSplit': 0.2, 'traintestSplit': 0.8, 
                  'Quincy': False, 'Alex': False, 'DataSeenPerEpoch': 4720}
    # print('Getting data')
    train_tasks,val_loader,test_loader,Params=utils.colabgetdata(Params)

   

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/content/utils.py:233: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xtest = torch.tensor(test_data).permute(0, 3, 1, 2)


In [ ]:
Params, trainacc, trainloss, valacc, valtopk = epoch.epochthrough(train_tasks,Params,val_loader)


In [ ]:
loss_rate = valacc[-1]
reduce = [True,False]
nwayss =[32,20,10,5]
for i in reduce:
    Params['Order']=i
    for tasknum in range(4): ## number of tasks inside each outer loop
        Params["number_of_tasks"] = int(2**tasknum)
        for InoutRatio in range(2): ## ratio of data in outer loop of task to one of the data in tasks right now it is 1:1, 2:1, 3:1
            Params['outerVSinner'] = InoutRatio+1
            for nwayys in nwayss:
                Params['nways']=nwayys
                Params['innerStep'] = .0005
                Params['MetaLR'] = .001
                for meta in range(5):## learning rates
                    Params['MetaLR'] *=(random.uniform(.25,1.25))
                    for innLR in range(5):
                    Params["innerStep"] *=(random.uniform(.25,1.25)
                    model = MODEL.ConvolutionalNeuralNetwork(Params)
                    model = model.to(device)
                    Atrain = 0
                    Ltrain  =0
                    Aval = 0
                    Akval = 0
                    for i in range(Params['epoch']):
                        Atrain, Ltrain = TRAIN.train(model, train_tasks,Params, val_loader)
                        Aval, Akval = utils.getvalErr(model, val_loader)
                        if ((i <= 4) and Aval < .2): ## not to waste time stop early
                             break
                        if Aval > loss_rate: ## if our val is better then what we had save it
                            print("found one better")
                            print(loss_rate)
                            print(Params)
                            dict = {'Training loss': Ltrain, "Training Error Rate": Atrain, "Validation topk": Akval,
                                    "Validation Error Rate": Aval}
                            tofile = {**dict, **Params}
                            with open('/content/drive/MyDrive/Project/BestSoFar.txt', 'a') as f:
                                print(tofile, file=f)
                                loss_rate = dict["Validation Error Rate"]